##<strong>KoBERT를 활용한 감정분류모델</strong>
###- 연속성 대화 + 단발성 대화 + 욕설 감지 혼합 데이터셋

* 데이터셋 설명
  - 연속성 대화 + 단발성 대화 + 혐오표현 추가 = total_train, total_test(전처리 파일에서 통합한 데이터셋)

    +) 감정분류 데이터셋
      - AI HUB 감성대화말뭉치 https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86 (연속성 대화)
      - AI HUB 한국어 단발성 대화

    +) 혐오표현 데이터셋
      - https://github.com/2runo/Curse-detection-data

In [ ]:
# 구글 드라이브 마운트 및 모듈 임포트
!pip install mxnet==1.6.0 gluonnlp==0.9.1
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=557726 sha256=71696e65834f77d8168f20613140f3ddf8c2851dd9f8573721c7a710ed3b732f
  Stored in directory: /root/.cache/pip/wheels/fc/5b/9c/3295bb07f7c5544a96303a48988707816f44a536e8e1413922
Successfully built gluonnlp
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# github에서 KoBERT 파일 임포트 및 모델 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# KoBERT의 tokenizer, model, vocabulary 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-tmtponn0/kobert-tokenizer_da8408bdf5e44d32a34a1b50254cc4e0
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-tmtponn0/kobert-tokenizer_da8408bdf5e44d32a34a1b50254cc4e0
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=959954be98513e23e8b1cdfad17bf988151fd3d8cf16d32a4f1385902fcfce67
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3arzmhs/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# GPU 설정
device = torch.device("cuda:0")

### 데이터셋 불러오기 및 전처리
- columns: Sentence(한국어 문장), Emotion(8가지)
- train과 test 데이터셋 이미 분리(test_size = 0.2)된 데이터셋
- 감정 라벨링 진행후, 문장 뒤에 라벨 삽입된 데이터셋 만들기

In [ ]:
# 데이터셋 불러오기
total_train = pd.read_csv('/content/drive/MyDrive/data/total_train_real.csv')
total_test = pd.read_csv('/content/drive/MyDrive/data/total_test_real.csv')

In [ ]:
# 데이터셋 전처리(total_train)

# emotion_dict 만들기위한 라벨링
mapping = {'슬픔':0, '중립':1, '불안':2, '당황':3, '분노':4, '기쁨':5, '혐오':6, '상처':7, '행복':8, '놀람':9, '공포':10}

In [ ]:
# 필요없는 컬럼 삭제
total_train.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
total_train

,Sentence,Emotion
0,얘네는 댓글만 보면 중국가수네,놀람
1,니들이 1~2년 쓰다가 고장나는건 폰을 개떡같이 쓰니까 그런거야 펴엉신들아ㅉㅉ앱등이 수준,혐오
2,항상 감사합니다 용규선수ㅠㅠㅠㅠ,행복
3,박근혜와 친박 개누리들이 버틴다면 세자릿수도 가능할듯,혐오
4,질싸?,혐오
...,...,...
57838,부모님이 내가 밖에 나가려고만 하면 간섭을 하셔. 속상해.,당황
57839,부모님은 나에게 관심도 없어. 내가 무슨 말을 하든 다 잊어버리는 것 같아서 짜증 나.,불안
57840,내가 지금 할 수 있는 건 아무것도 없어. 아내는 계속 아르바이트라도 알아보라고 해.,슬픔
57841,결혼 후 독립에 대한 양가 부모님의 생각이 달라. 어떻게 하면 좋을지 모르겠어.,당황


In [ ]:
# 컬럼으로 추가
total_train.insert(2, 'Emotion_label', total_train.Emotion.map(mapping))
total_train.tail()

,Sentence,Emotion,Emotion_label
57838,부모님이 내가 밖에 나가려고만 하면 간섭을 하셔. 속상해.,당황,3
57839,부모님은 나에게 관심도 없어. 내가 무슨 말을 하든 다 잊어버리는 것 같아서 짜증 나.,불안,2
57840,내가 지금 할 수 있는 건 아무것도 없어. 아내는 계속 아르바이트라도 알아보라고 해.,슬픔,0
57841,결혼 후 독립에 대한 양가 부모님의 생각이 달라. 어떻게 하면 좋을지 모르겠어.,당황,3
57842,내가 아내한테 정말 잘해주고 있는 게 맞는지 의문이 들어. 속상해.,당황,3


In [ ]:
# total_test 데이터셋도 라벨링 작업
total_test.insert(2, 'Emotion_label', total_test.Emotion.map(mapping))
total_test.tail()

,Unnamed: 0,Sentence,Emotion_label,Emotion
11113,6636,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,3,당황
11114,6637,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,3,당황
11115,6638,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,3,당황
11116,6639,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,3,당황
11117,6640,친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.,3,당황


In [ ]:
total_test.drop('Unnamed: 0', axis=1, inplace=True)
total_test.head()

,Sentence,Emotion_label,Emotion
0,지디게이 죽여 없애야됨,6,혐오
1,친일파들을 죽여라~,6,혐오
2,착각들 하는데 얼굴이 큰거랑 머리가 큰거는 다른뎅,1,중립
3,개 씝 극우 쪽빠리 수장 새끼,6,혐오
4,삼성이 돈을 조금더 벌려고 싸구려 중국산 밧데리 장착했다가 개망신 당하는구나!,4,분노


In [ ]:
# ['문장','라벨'] 형태로 데이터 변환
dataset_train = []
for q, label in zip(total_train['Sentence'], total_train['Emotion_label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_train.append(data)

In [ ]:
dataset_train[:10]

[['얘네는 댓글만 보면 중국가수네', '9'],
 ['니들이 1~2년 쓰다가 고장나는건 폰을 개떡같이 쓰니까 그런거야 펴엉신들아ㅉㅉ앱등이 수준', '6'],
 ['항상 감사합니다 용규선수ㅠㅠㅠㅠ', '8'],
 ['박근혜와 친박 개누리들이 버틴다면 세자릿수도 가능할듯', '6'],
 ['질싸?', '6'],
 ['와 직장인으로서 7천만 연봉이었는데 지금 다 버리고 1천만 연봉 배우..', '9'],
 [' 종합상사를지원해서 남미. 유럽. 중앙아시아. 등으로', '1'],
 [' 니들 언젠간사지가찢어지는고통느낄테니까', '9'],
 ['아이폰 껍데기에 삼성하드웨어 엘지베터리를 ios로 구동하면 ..', '1'],
 ['노인운전자 위험해요', '10']]

In [ ]:
dataset_test = []
for q, label in zip(total_test['Sentence'], total_test['Emotion_label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_test.append(data)

In [ ]:
dataset_test[:10]

[['지디게이 죽여 없애야됨', '6'],
 ['친일파들을 죽여라~', '6'],
 ['착각들 하는데 얼굴이 큰거랑 머리가 큰거는 다른뎅', '1'],
 ['개 씝 극우 쪽빠리 수장 새끼', '6'],
 ['삼성이 돈을 조금더 벌려고 싸구려 중국산 밧데리 장착했다가 개망신 당하는구나!', '4'],
 ['그래서 어디서부터 어떻게 배우고 해야하는지 정말고민이네요ㅠㅠㅠ', '10'],
 ['올해는 경쟁자도 없는데 무라카미 하루키에게 주면 될걸 괜히 미국 딴따라에게 줘서 무라카미 하루키를 물먹였다', '6'],
 ['아니 왜 달의 연인 댓글을 왜 여기다 쓰고 ㅈI 랄이야', '4'],
 ['참 기자양반 가지가지로 조회수 늘릴려고 하네... ㅎㅎㅎ', '6'],
 ['박근혜 최순실 광화문 한복한에 놓고 장성택 꼴내고싶다', '4']]

#### KoBERT 모델 적용을 위한 데이터셋 정제
- class BERTDataset
- 토큰화, 정수 인코딩, 패딩

In [ ]:
# KoBERT 모델 사용을 위한 클래스
# django에 학습한 모델을 로드할 때 이 클래스들(BERTDataset, BERTClassifier)은 manage.py에 올려두고 Apps.py에서 로드



class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')



class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,  # 분류갯수
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 파라미터 정립
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
# torch 형식으로 변환
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### KoBERT 모델 구현하기

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# GPU 세팅 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# 모델 학습 진행
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-21-9ff322c5085d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/904 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.4528965950012207 train acc 0.109375
epoch 1 batch id 201 loss 1.9425240755081177 train acc 0.1896766169154229
epoch 1 batch id 401 loss 1.5448652505874634 train acc 0.2894326683291771
epoch 1 batch id 601 loss 1.8176029920578003 train acc 0.32435524126455906
epoch 1 batch id 801 loss 1.4283243417739868 train acc 0.3742197253433209
epoch 1 train acc 0.38797406407253165


<ipython-input-21-9ff322c5085d>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/174 [00:00<?, ?it/s]

...

epoch 9 test acc 0.44108414542728636


  0%|          | 0/904 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 1.7444998025894165 train acc 0.546875
epoch 10 batch id 201 loss 0.20172010362148285 train acc 0.8564210199004975
epoch 10 batch id 401 loss 0.21167558431625366 train acc 0.9018079800498753
epoch 10 batch id 601 loss 1.5580699443817139 train acc 0.8681104409317804
epoch 10 batch id 801 loss 0.8002288937568665 train acc 0.8352059925093633
epoch 10 train acc 0.8271475902307825


  0%|          | 0/174 [00:00<?, ?it/s]

epoch 10 test acc 0.5656546726636682


#### 모델 평가
- 모델 평가(정확도,...)
- 새로운 문장을 입력해서 테스트(감정 분류 잘하는지)

In [ ]:
# 새로운 문장 테스트

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 1:
                test_eval.append("중립이")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("당황이")
            elif np.argmax(logits) == 4:
                test_eval.append("분노가")
            elif np.argmax(logits) == 5:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 7:
                test_eval.append("상처가")
            elif np.argmax(logits) == 8:
                test_eval.append("행복이")
            elif np.argmax(logits) == 9:
                test_eval.append("놀람이")
            else:
              test_eval.append("공포가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 학습모델 저장
PATH = '/content/drive/MyDrive/data/'
torch.save(model, PATH + 'KoBERT_11emotions_final.pt')#전체모델저장
torch.save(model.state_dict(), PATH + 'model_state_dict_final.pt')# 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 친구들이 나만 빼고 먼저 급식실에 갔어
>> 입력하신 내용에서 당황이 느껴집니다.


하고싶은 말을 입력해주세요 : 좀 기다려주지 자기들끼리만 먹으러가고
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 난 맨날 기다려줬는데 걔네들은 안그러더라
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 몰라 이제 다 짜증나
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이미 지들끼리 먹으러가고 나중에 와서 챙겨주는 척 재수없어
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 학교에 왜 다녀야 하는거야 ㅅㅂ
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 아 학교 가기 싫다
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 다싫어 그냥
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 배고프다
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 집에 마카롱 남은 거 먹어야지
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 마카롱 맛있다!!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 어제 고양이들이 갑자기 거실에서 우다다 도망갔는데 무서웠어
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 집에 귀신이 있으면 어쩌지
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : ㅈ;ㅂㅇ[ 귀신이 있는 것 같아
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 0
